In [2]:
from auto_login import get_curr_path,get_credentials,auto_login,generate_access_token
import os 
import logging 
from kiteconnect import KiteConnect
import datetime as dt
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 
import ta

In [3]:
folder_name = 'api_keys'
curr_dir = get_curr_path(folder_name)

file_name = 'credentials.txt'
api_key,api_secret,user_name,pwd,totp_key = get_credentials(curr_dir,file_name)

request_token = auto_login(api_key,user_name,pwd,totp_key)

access_token = generate_access_token(request_token,api_key,api_secret)

In [4]:
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token=access_token)
instrument_dump = kite.instruments('NSE')
instrument_df = pd.DataFrame(instrument_dump)
instrument_df.to_csv('NSE_instruments.csv',index=False)

In [5]:
instrument_df.head()

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,256265,1001,NIFTY 50,NIFTY 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE
1,256777,1003,NIFTY MIDCAP 100,NIFTY MIDCAP 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
2,260105,1016,NIFTY BANK,NIFTY BANK,0.0,,0.0,0.0,0,EQ,INDICES,NSE
3,260617,1018,NIFTY 100,NIFTY 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
4,257033,1004,NIFTY DIV OPPS 50,NIFTY DIV OPPS 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE


In [6]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1

In [7]:
def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(instrument_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data

In [8]:
infy_df = fetchOHLC('INFY','5minute',5)
infy_df.to_csv('infy.csv')

In [9]:
infy_df.head()

,open,high,low,close,volume
date,,,,,
2024-05-31 09:15:00+05:30,1409.80,1414.15,1405.10,1413.30,640646
2024-05-31 09:20:00+05:30,1414.00,1414.75,1409.70,1411.20,564481
2024-05-31 09:25:00+05:30,1411.25,1415.45,1410.40,1413.45,484983
2024-05-31 09:30:00+05:30,1412.95,1414.70,1412.60,1414.30,610608
2024-05-31 09:35:00+05:30,1414.30,1414.30,1407.65,1409.80,239481


In [10]:
df = infy_df.copy()

In [11]:
# The True Range is the largest of the following:

# The Current Period High minus (-) Current Period Low
# The Absolute Value (abs) of the Current Period High minus (-) The Previous Period Close
# The Absolute Value (abs) of the Current Period Low minus (-) The Previous Period Close
# true range = max[(high - low), abs(high - previous close), abs (low - previous close)]


# *Absolute Value is used because the ATR does not measure price direction, 
# only volatility. Therefore there should be no negative numbers. 
# *Once you have the True Range, the Average True Range can be plotted. 
# By default on TradingView the ATR is a Relative Moving Average (RMA) of the True Range, 
# but the smoothing type can be changed to SMA, EMA or WMA in the settings.
# how to rma calculate based on this 

def compute_true_range(df,n):
    df['high-low'] = df['high'] - df['low']
    df['high-prev_close'] = abs(df['high'] - df['close'].shift(1))
    df['low-prev_close'] = abs(df['low'] - df['close'].shift(1))
    df['true_range'] = df[['high-low','high-prev_close','low-prev_close']].max(axis=1,skipna=False)
    # df['atr'] = df['true_range'].rolling(n).mean()
    return df


def compute_atr(df,n):
    alpha = 1/n
    df['temp_ma_init'] = df['true_range'].rolling(n).mean()
    df.dropna(inplace=True)
    for i in range(1,len(df)):
        if i == 1:
            df.loc[df.index[i],'atr'] = df.loc[df.index[i],'temp_ma_init']
        else:
            df.loc[df.index[i],'atr'] = alpha*df.loc[df.index[i],'true_range'] + (1 - alpha)*df.loc[df.index[i-1],'atr']
    return df



    

In [12]:
df = compute_true_range(df,14)
df = compute_atr(df,14)
df


,open,high,low,close,volume,high-low,high-prev_close,low-prev_close,true_range,temp_ma_init,atr
date,,,,,,,,,,,
2024-05-31 10:25:00+05:30,1404.35,1408.00,1404.30,1406.20,257704,3.70,3.65,0.05,3.70,3.925000,NaN
2024-05-31 10:30:00+05:30,1407.15,1410.55,1406.30,1409.40,300823,4.25,4.35,0.10,4.35,3.875000,3.875000
2024-05-31 10:35:00+05:30,1409.35,1412.65,1408.00,1411.70,394769,4.65,3.25,1.40,4.65,3.846429,3.930357
2024-05-31 10:40:00+05:30,1411.15,1412.40,1410.40,1410.95,254669,2.00,0.70,1.30,2.00,3.839286,3.792474
2024-05-31 10:45:00+05:30,1410.90,1412.40,1409.70,1411.45,280132,2.70,1.45,1.25,2.70,3.557143,3.714441
...,...,...,...,...,...,...,...,...,...,...,...
2024-06-05 15:05:00+05:30,1431.55,1433.05,1430.55,1433.00,258006,2.50,1.45,1.05,2.50,3.382143,3.293949
2024-06-05 15:10:00+05:30,1433.00,1434.00,1429.40,1430.15,286080,4.60,1.00,3.60,4.60,3.442857,3.387238
2024-06-05 15:15:00+05:30,1430.15,1431.00,1426.45,1430.05,356676,4.55,0.85,3.70,4.55,3.617857,3.470293
